## LLaMA ChatInterface Gradio Demo

You must have llama model checkpoints to use this notebook. Pls see the [LLaMA](https://github.com/meta-llama/llama-models) documentation and request-download model checkpoints from huggingface..

In [ ]:
!pip install transformers --upgrade
!pip install gradio --upgrade
!pip install accelerate --upgrade

In [2]:
import os
# no nvlink
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
from transformers import AutoTokenizer
import transformers
import gradio as gr
import torch

# path to the model
model = "/data/llm/llama/Meta-Llama-3.1-8B-Instruct"
model = "/data/llm/llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


def echo(message, history, max_new_tokens=100):
    print("History: ", history)
    # pre-pend the history to message
    past_messages = ""
    if history:
        past_messages = history[0][-1]
    message = past_messages + " " + message
    print("Message", message)
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        truncation = False,
        max_new_tokens=max_new_tokens,
    )

    answer = sequences[0]['generated_text']
    answer = answer[len(message):]
    # find "\n\n"
    # remove string after the index
    index = answer.find("\n\n")
    if index != -1:
        answer = answer[:index]
    
    return answer


title = model.split("/")[-1] + " Chat"
demo = gr.ChatInterface(
    fn=echo,
    examples=["What are the planets in the solar system?", "Translate this to Tagalog: Hello, how are you?"],
    title=title,
).launch(inbrowser=True)

